## Problem Statement

To identify and classify images as cats or dogs

## Learning Objectives

At the end of the experiment, you will be able to :

* Load and prepare images for the model using Pytorch
* Develop a CNN model and improve model performance

## Dataset

The Dogs and Cats dataset is a standard computer vision dataset that involves classifying photos as either containing a dog or cat. The train folder contains ~22k images of dogs and cats. Each image in this folder has the label as part of the filename. The test folder contains ~2k images, named according to a numeric id.

In [ ]:
#@title Run this cell to download the dataset

from IPython import get_ipython
ipython = get_ipython()

notebook="U3_MH1_CNN_with_Pytorch" #name of the notebook
Answer = "This notebook is graded by mentors on the day of hackathon"
def setup():
#
   ipython.magic("sx wget http://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Cat_Dog_data_B17.zip")
   ipython.magic("sx unzip -qq Cat_Dog_data_B17.zip")
   print ("Setup completed successfully")
   return

setup()

Setup completed successfully


In [ ]:
# Import Libraries
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

In [ ]:
image_size = (128,128)

transformations =  transforms.Compose([transforms.Resize(image_size),transforms.Grayscale(num_output_channels=1),transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])# YOUR CODE HERE for defining Transformation for an image

In [ ]:
batch_size = 512

train_set = datasets.ImageFolder('/content/Cat_Dog_data/train', transform = transformations)

trainloader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size = batch_size)

In [ ]:
for (X_train, y_train) in trainloader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([100, 1, 128, 128]) type: torch.FloatTensor
y_train: torch.Size([100]) type: torch.LongTensor


In [ ]:

labels =[]
features = []
cats = 0
dogs = 0
for X,y in zip(X_train, y_train):
  # Getting unique labels
  if cats<5 or dogs<5:
    if cats<5 and y==0:
      labels.append(y)
      features.append(X)
      cats = cats + 1
    if dogs<5 and y==1:
      labels.append(y)
      features.append(X)
      dogs = dogs + 1
  else:
    break

labels_map = {0 : 'cat', 1 : 'dog'};
pltsize=1
plt.figure(figsize=(7,7))
for i in range(10):
    plt.subplot(4,3, i+1)
    plt.axis('off')
    # Convert the tensor to numpy for displaying the image
    plt.imshow(features[i].numpy().reshape(128,128), cmap="gray")
    plt.title('Class: '+labels_map[int(str(labels[i].numpy()))])

NameError: name 'X_train' is not defined

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Sample Convolution Layer 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2) # output size of the first convolutional layer is 16*128*128 ((input_size+2*padding-kernel_size)/stride)+1
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=4) # Maxpooling reduces the size by kernel size. After Maxpooling the output size is 16*32*32


        # YOUR CODE HERE for defining more number of Convolutional layers with Maxpool as required (Hint: Use at least 3 convolutional layers for better performance)
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1, padding=2) # output size of the first convolutional layer is 16*32*32
        self.bn2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Maxpooling reduces the size by kernel size. After Maxpooling the output size is 32*16*16

        self.cnn3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1, padding=2) # output size of the first convolutional layer is 16*16*16
        self.bn3 = nn.BatchNorm2d(16)
        self.relu3 = nn.ReLU()
        # Sample Maxpool for the Convolutional Layer 1
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2) # Maxpooling reduces the size by kernel size. After Maxpooling the output size is 64*8*8





        # YOUR CODE HERE for defining the Fully Connected Layer and also define LogSoftmax

        self.fc1 = nn.Linear(16*8*8,128)
        self.fc3 = nn.Linear(128,32)
        self.fc4 = nn.Linear(32,2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Convolution Layer 1 and Maxpool
        #print(f"1 {x.shape}")
        out = self.cnn1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.maxpool1(out)
        #print(f"2 {out.shape}")

        # YOUR CODE HERE for the Convolutional Layers and Maxpool based on the defined Convolutional layers
        out = self.cnn2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        #print(f"3 {out.shape}")

        out = self.cnn3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)
        #print(f"4 {out.shape}")




        # YOUR CODE HERE for flattening the output of the final pooling layer to a vector. Flattening is simply arranging the 3D volume of numbers into a 1D vector

        out = out.view(-1,16*8*8)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc3(out))
        out = self.fc4(out)
        out = self.softmax(out)
        # YOUR CODE HERE for returning the output of LogSoftmax after applying Fully Connected Layer
        return out


In [ ]:
# To run the training on GPU --Rushabh
print(torch.cuda.is_available())

device  =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [ ]:
model = CNNModel()
model = model.to(device)
print(model)

#criterion = # YOUR CODE HERE : Explore and declare loss function

#optimizer = # YOUR CODE HERE : Explore on the optimizer and define with the learning rate
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bia

In [ ]:
val_set = datasets.ImageFolder('/content/Cat_Dog_data/test',transform = transformations)
# YOUR CODE HERE for the DataLoader
testloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size = batch_size)

In [ ]:
# YOUR CODE HERE. This will take time --Anupreksha

# Record loss and accuracy of the train dataset
epoch = 15

# keeping the network in train mode
model.train()
train_losses,  train_accuracy = [], []

# Loop for no of epochs
for e in range(epoch):
    model.train()
    train_loss = 0
    correct = 0
    # Iterate through all the batches in each epoch
    for images, labels in trainloader:

      # Convert the image and label to gpu for faster execution
      images = images.to(device)
      labels = labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Passing the data to the model (Forward Pass)
      outputs = model(images)

      # Calculating the loss
      loss = criterion(outputs, labels)
      train_loss += loss.item()

      # Performing backward pass (Backpropagation)
      loss.backward() # y' = mx+c layer1 w1-> l2 w2-> output (loss) dW1 db1

      # optimizer.step() updates the weights accordingly
      optimizer.step()

      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()

    # Accuracy calculation
    train_losses.append(train_loss/len(train_set))
    train_accuracy.append(100 * correct/len(train_set))
    print('epoch: {}, Train Loss:{:.6f} Train Accuracy: {:.2f} '.format(e+1,train_losses[-1], train_accuracy[-1]))
    model.eval()
    Test_accuracy = 0
    for images,labels in testloader:

      # Convert the images and labels to gpu for faster execution
      images = images.to(device)
      labels = labels.to(device)

      # Do the forward pass
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      Test_accuracy += (predicted == labels).sum().item()
    Accuracy = 100 * Test_accuracy / len(val_set)
    print("Accuracy of Test Data is", Accuracy)

epoch: 1, Train Loss:0.001156 Train Accuracy: 68.96 
Accuracy of Test Data is 53.691931540342296
epoch: 2, Train Loss:0.001002 Train Accuracy: 75.65 
Accuracy of Test Data is 73.49633251833741
epoch: 3, Train Loss:0.000906 Train Accuracy: 78.63 
Accuracy of Test Data is 78.2885085574572
epoch: 4, Train Loss:0.000823 Train Accuracy: 81.11 
Accuracy of Test Data is 74.1320293398533
epoch: 5, Train Loss:0.000756 Train Accuracy: 82.79 
Accuracy of Test Data is 77.75061124694376
epoch: 6, Train Loss:0.000728 Train Accuracy: 83.82 
Accuracy of Test Data is 80.29339853300733
epoch: 7, Train Loss:0.000676 Train Accuracy: 85.15 
Accuracy of Test Data is 82.10268948655256
epoch: 8, Train Loss:0.000633 Train Accuracy: 86.33 
Accuracy of Test Data is 81.9559902200489
epoch: 9, Train Loss:0.000617 Train Accuracy: 86.39 
Accuracy of Test Data is 78.58190709046455
epoch: 10, Train Loss:0.000567 Train Accuracy: 87.88 
Accuracy of Test Data is 61.613691931540345
epoch: 11, Train Loss:0.000567 Train Acc

In [ ]:
val_set = datasets.ImageFolder('/content/Cat_Dog_data/test',transform = transformations)
# YOUR CODE HERE for the DataLoader
testloader = torch.utils.data.DataLoader(val_set, shuffle=True, batch_size = batch_size)

In [ ]:
model.eval()
# YOUR CODE HERE for calculating the accuracy
Test_accuracy = 0

# Iterate through all the batches in each epoch
for images,labels in testloader:

    # Convert the images and labels to gpu for faster execution
    images = images.to(device)
    labels = labels.to(device)

    # Do the forward pass
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    Test_accuracy += (predicted == labels).sum().item()
print("Total correct predictions: ", Test_accuracy)
Accuracy = 100 * Test_accuracy / len(val_set)
print("Accuracy of Test Data is", Accuracy)

Total correct predictions:  1769
Accuracy of Test Data is 86.50366748166259
